In [1]:
import stockutils, basicutils, matchutils, stringparsing, config
import re

In [2]:
class StockData:
    def __init__(self, update = False):
        self.company_table = stockutils.company_table(update = update)
    
    @basicutils.memo
    def lookup(self, search_info = ('symbol', 'name', 'se'), num_results = 5, **conditions):
        """Converts company_name to ticker symbol. Returns a tuple of (symbol, company name) 
        or returns an empty table if company or ticker is not found."""
        initial = conditions.copy()
        if isinstance(search_info, str):
            current_search = [search_info[:]]
        else:
            current_search = list(search_info)
        order = []
        if 'name' in conditions:
            fuzzy_names = []
            if not isinstance(conditions['name'], list):
                conditions['name'] = [conditions['name']] 
            for name in conditions['name']:
                fuzzy_names.extend([self.company_table[match] 
                                    for match in matchutils.fuzzy_search(name, self.company_table.keys(), num_results = num_results)])
            order = conditions['name'] = fuzzy_names
            current_search.append('name')
        elif 'symbol' in conditions:
            order = conditions['symbol'] if isinstance(conditions['symbol'], list) else [conditions['symbol']]
            current_search.append('symbol')
        results = stockutils.sql_search(config.sql_path, config.sql_table_name, current_search, **conditions)
        if order:
            sorted_results = [item[:-1] for identifier in order for item in results if identifier == item[-1]]
            return sorted_results or [("No results found matching following conditions: ", str(initial))]
        return results or [("No results found matching following conditions: ", str(initial))]

    def parse_companies(self, sentence):
        return self.lookup(name = stringparsing.fuzzy_ner(sentence,self.company_table.keys()))
            
    
            
#       def stock_price(self, ticker, date):
            
        
        

In [3]:
basicutils.import_cache('data/cached_searches.json')

In [4]:
basicutils.clear_cache('data/cached_searches.json')

In [5]:
w = StockData()

In [6]:
w.parse_companies("Indonesia: J.P. Morgan was tasked to find buyers, but instead said 'sell'")

[('JPM', 'J P Morgan Chase & Co', 'nyse'),
 ('IF', 'Aberdeen Indonesia Fund, Inc.', 'amex'),
 ('TLK', 'PT Telekomunikasi Indonesia, Tbk', 'nyse')]

In [7]:
w.parse_companies('Amazon to open first Manhattan bookstore')

[('AMZN', 'Amazon.com, Inc.', 'nasdaq'),
 ('MANH', 'Manhattan Associates, Inc.', 'nasdaq'),
 ('LOAN', 'Manhattan Bridge Capital, Inc', 'nasdaq')]

In [8]:
w.parse_companies('Intel to expand sales in Europe')

[('INTC', 'Intel Corporation', 'nasdaq')]

In [9]:
w.lookup(ipoyear=(list(range(1990,1991))), se = 'amex')

[('IF', 'Aberdeen Indonesia Fund, Inc.', 'amex')]

In [10]:
w.parse_companies('Apple to maintain Asian production lines.')

[('APLE', 'Apple Hospitality REIT, Inc.', 'nyse'),
 ('AAPL', 'Apple Inc.', 'nasdaq')]

In [11]:
w.parse_companies("Moody's to downgrade shipping companies' ratings.")

[('MCO', "Moody's Corporation", 'nyse')]

In [12]:
w.parse_companies('Apple pulls New York Times app from China store')

[('NYT', 'New York Times Company (The)', 'nyse'),
 ('APLE', 'Apple Hospitality REIT, Inc.', 'nyse'),
 ('AAPL', 'Apple Inc.', 'nasdaq')]

In [13]:
w.parse_companies("Macy's is closing 63 stores this spring. Check to see if your local Macy's is on the list: http://on.wsj.com/2hSwfTr")

[('M', "Macy's Inc", 'nyse'),
 ('CHKP', 'Check Point Software Technologies Ltd.', 'nasdaq'),
 ('CHEK', 'Check-Cap Ltd.', 'nasdaq')]

In [14]:
w.lookup(search_info = ['marketcap', 'name'], symbol = 'BDX')

[('$38.42B', 'Becton, Dickinson and Company')]

In [15]:
matchutils.fuzzy_search('63', w.company_table.keys())

[]

In [16]:
w.parse_companies('Hedge-fund manager Edward Lampert has pumped an additional $1 billion into his yearslong effort to revive Sears')

[('EW', 'Edwards Lifesciences Corporation', 'nyse'),
 ('SHLD', 'Sears Holdings Corporation', 'nasdaq'),
 ('SHOS', 'Sears Hometown and Outlet Stores, Inc.', 'nasdaq')]

In [17]:
w.lookup(name = 'Sears')

[('SHLD', 'Sears Holdings Corporation', 'nasdaq'),
 ('SHOS', 'Sears Hometown and Outlet Stores, Inc.', 'nasdaq'),
 ('SRSC', 'Sears Canada Inc. ', 'nasdaq')]